<a href="https://colab.research.google.com/github/Marglodis/challenge2-data-science-LATAM/blob/main/TelecomX_LATAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Challenge Telecom X: Análisis de evasión de clientes

#📌 Extracción

#🔧 Transformación

#📊 Carga y análisis

#📄Informe final